# Read Data

In [1]:
import os
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dropout, Dense
from keras.models import load_model
import numpy as np
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data_dir = '.\\data'
save_dir = '.\\model'

In [3]:
file_paths = [data_dir + '\\' + filename for filename in os.listdir(data_dir)]

lyrics_list = []
for path in file_paths:
    with open(path, 'r') as file:
        lyrics_list.append(file.read())

lyrics_concatenated = '\n'.join(lyrics_list)

In [4]:
characters = list(lyrics_concatenated)
character_set = set(characters)

vocab_size = len(character_set)

In [5]:
current_index = 0
char_to_index = {}
index_to_char = {}
for character in character_set:
    char_to_index[character] = current_index
    index_to_char[current_index] = character
    current_index += 1

# Write char to index dictionary to file

In [6]:
import json
  
save_dir = '.\\model'
file_name = 'char_to_index.json'
json = json.dumps(char_to_index)
f = open(save_dir + '\\' + file_name, 'w')
f.write(json)
f.close()

# Convert all lyrics to encodings

In [7]:
lyrics_encoded = [char_to_index[x] for x in characters]

# Initial Configurations

In [8]:
sequence_lengths = [32, 64]
batch_sizes = [16, 32]
embedding_sizes = [256, 512]
lstm_units = [128, 256]
lstm_layers = 3
dropout_rate = 0.2
epochs = 100

combinations = list(zip(sequence_lengths, batch_sizes, embedding_sizes, lstm_units))

# Generator for Train Data

In [9]:
def generate_batched_train_data(batch_size, sequence_length):
    current_index = 0
    total_size = len(characters)
    num_steps = int(total_size / (sequence_length * batch_size))
    
    for _ in range(num_steps):
        x_batched = np.zeros((batch_size, sequence_length))
        y_batched = np.zeros((batch_size, sequence_length, vocab_size))
        
        for i in range(batch_size):
            for j in range(sequence_length):
                x_batched[i, j] = lyrics_encoded[current_index]
                y_batched[i, j, lyrics_encoded[current_index + 1]] = 1
                current_index += 1
        
        yield x_batched, y_batched

# build model

In [11]:
def build_model(embedding_size, lstm_units_count, sequence_length):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, batch_input_shape = (batch_size, sequence_length)))

    for i in range(lstm_layers):
        model.add(LSTM(lstm_units_count, return_sequences = True, stateful = True))
        model.add(Dropout(rate = dropout_rate))

    model.add(TimeDistributed(Dense(vocab_size, activation = 'softmax')))
    
    return model

# Train the model

In [12]:
accuracies = []
losses = []

for sequence_length, batch_size, embedding_size, lstm_units_count in combinations:
    print('Training model for sequence length: {}, batch_size: {}, embedding_size: {}, lstm_units_count: {}'.format(sequence_length, batch_size, embedding_size, lstm_units_count))
    
    model = build_model(embedding_size, lstm_units_count, sequence_length)
    
    model.compile(optimizer = 'Adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
        
    for epoch in range(epochs):
        step_count = 0
        for x_train_batched, y_train_batched in generate_batched_train_data(batch_size, sequence_length):
            loss, accuracy = model.train_on_batch(x_train_batched, y_train_batched)

            if step_count % 10 == 0:
                print('Epoch: {}, step_count: {}, loss: {}, accuracy: {}'.format(epoch, step_count, loss, accuracy))

            step_count += 1
    
    print('Saving model for sequence length: {}, batch_size: {}, embedding_size: {}, lstm_units_count: {}'.format(sequence_length, batch_size, embedding_size, lstm_units_count))
    save_file_name = 'Sequence length={} batch_size={} embedding_size={} lstm_units_count={}.model.h5'.format(sequence_length, batch_size, embedding_size, lstm_units_count)
    model.save(save_dir + '\\' + save_file_name)
    model.save_weights(save_dir + '\\' + save_file_name)
    losses.append(loss)
    accuracies.append(accuracy)

print(accuracies)
print(losses)

Training model for sequence length: 32, batch_size: 16, embedding_size: 256, lstm_units_count: 128
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch: 0, step_count: 0, loss: 4.159358978271484, accuracy: 0.0078125
Epoch: 0, step_count: 10, loss: 3.4233860969543457, accuracy: 0.17578125
Epoch: 0, step_count: 20, loss: 3.28476619720459, accuracy: 0.173828125
Epoch: 0, step_count: 30, loss: 3.155109405517578, accuracy: 0.1640625
Epoch: 0, step_count: 40, loss: 3.203090190887451, accuracy: 0.15625
Epoch: 0, step_count: 50, loss: 3.2200865745544434, accuracy: 0.16015625
Epoch: 0, step_count: 60, loss: 3.110851764678955, accuracy: 0.169921875
Epoch: 0, step_count: 70, loss: 3.1390762329101562, accuracy: 0.154296875
Epoch: 0, step_count: 80, loss: 3.2191829681396484, accuracy: 0.150390625
Epoch: 0, step_co

Epoch: 7, step_count: 90, loss: 2.314358711242676, accuracy: 0.3828125
Epoch: 7, step_count: 100, loss: 2.161266803741455, accuracy: 0.375
Epoch: 7, step_count: 110, loss: 2.034342050552368, accuracy: 0.44921875
Epoch: 7, step_count: 120, loss: 2.2396128177642822, accuracy: 0.38671875
Epoch: 8, step_count: 0, loss: 2.093764305114746, accuracy: 0.41796875
Epoch: 8, step_count: 10, loss: 2.0555217266082764, accuracy: 0.390625
Epoch: 8, step_count: 20, loss: 2.406613826751709, accuracy: 0.359375
Epoch: 8, step_count: 30, loss: 2.068483352661133, accuracy: 0.4140625
Epoch: 8, step_count: 40, loss: 2.2132787704467773, accuracy: 0.400390625
Epoch: 8, step_count: 50, loss: 2.1936042308807373, accuracy: 0.36328125
Epoch: 8, step_count: 60, loss: 2.173666000366211, accuracy: 0.3984375
Epoch: 8, step_count: 70, loss: 2.1531801223754883, accuracy: 0.404296875
Epoch: 8, step_count: 80, loss: 2.306807041168213, accuracy: 0.361328125
Epoch: 8, step_count: 90, loss: 2.218484401702881, accuracy: 0.390

Epoch: 16, step_count: 40, loss: 1.9257361888885498, accuracy: 0.4609375
Epoch: 16, step_count: 50, loss: 1.8924047946929932, accuracy: 0.455078125
Epoch: 16, step_count: 60, loss: 1.8548563718795776, accuracy: 0.43359375
Epoch: 16, step_count: 70, loss: 1.844526767730713, accuracy: 0.48046875
Epoch: 16, step_count: 80, loss: 1.9954417943954468, accuracy: 0.439453125
Epoch: 16, step_count: 90, loss: 1.8868129253387451, accuracy: 0.451171875
Epoch: 16, step_count: 100, loss: 1.86051607131958, accuracy: 0.4609375
Epoch: 16, step_count: 110, loss: 1.7112085819244385, accuracy: 0.49609375
Epoch: 16, step_count: 120, loss: 1.9215995073318481, accuracy: 0.435546875
Epoch: 17, step_count: 0, loss: 1.790876865386963, accuracy: 0.494140625
Epoch: 17, step_count: 10, loss: 1.663356065750122, accuracy: 0.537109375
Epoch: 17, step_count: 20, loss: 1.978172779083252, accuracy: 0.3984375
Epoch: 17, step_count: 30, loss: 1.79041588306427, accuracy: 0.490234375
Epoch: 17, step_count: 40, loss: 1.90440

Epoch: 24, step_count: 110, loss: 1.5876269340515137, accuracy: 0.55859375
Epoch: 24, step_count: 120, loss: 1.7949050664901733, accuracy: 0.45703125
Epoch: 25, step_count: 0, loss: 1.682002305984497, accuracy: 0.525390625
Epoch: 25, step_count: 10, loss: 1.5053216218948364, accuracy: 0.580078125
Epoch: 25, step_count: 20, loss: 1.8083105087280273, accuracy: 0.46484375
Epoch: 25, step_count: 30, loss: 1.688481330871582, accuracy: 0.505859375
Epoch: 25, step_count: 40, loss: 1.7430272102355957, accuracy: 0.501953125
Epoch: 25, step_count: 50, loss: 1.7188503742218018, accuracy: 0.478515625
Epoch: 25, step_count: 60, loss: 1.6956090927124023, accuracy: 0.5
Epoch: 25, step_count: 70, loss: 1.7215150594711304, accuracy: 0.4921875
Epoch: 25, step_count: 80, loss: 1.8208110332489014, accuracy: 0.4921875
Epoch: 25, step_count: 90, loss: 1.726672887802124, accuracy: 0.509765625
Epoch: 25, step_count: 100, loss: 1.7385671138763428, accuracy: 0.4921875
Epoch: 25, step_count: 110, loss: 1.5793488

Epoch: 33, step_count: 50, loss: 1.664792537689209, accuracy: 0.48828125
Epoch: 33, step_count: 60, loss: 1.6701096296310425, accuracy: 0.48828125
Epoch: 33, step_count: 70, loss: 1.6240980625152588, accuracy: 0.51953125
Epoch: 33, step_count: 80, loss: 1.6991249322891235, accuracy: 0.48046875
Epoch: 33, step_count: 90, loss: 1.580559253692627, accuracy: 0.509765625
Epoch: 33, step_count: 100, loss: 1.6356289386749268, accuracy: 0.509765625
Epoch: 33, step_count: 110, loss: 1.4866523742675781, accuracy: 0.556640625
Epoch: 33, step_count: 120, loss: 1.7539564371109009, accuracy: 0.462890625
Epoch: 34, step_count: 0, loss: 1.635225772857666, accuracy: 0.529296875
Epoch: 34, step_count: 10, loss: 1.3586833477020264, accuracy: 0.603515625
Epoch: 34, step_count: 20, loss: 1.7340714931488037, accuracy: 0.47265625
Epoch: 34, step_count: 30, loss: 1.6293666362762451, accuracy: 0.533203125
Epoch: 34, step_count: 40, loss: 1.6270257234573364, accuracy: 0.533203125
Epoch: 34, step_count: 50, loss

...

Epoch: 93, step_count: 50, loss: 1.3021650314331055, accuracy: 0.591796875
Epoch: 93, step_count: 60, loss: 1.2987902164459229, accuracy: 0.595703125
Epoch: 93, step_count: 70, loss: 1.2906602621078491, accuracy: 0.611328125
Epoch: 93, step_count: 80, loss: 1.3190984725952148, accuracy: 0.599609375
Epoch: 93, step_count: 90, loss: 1.1219799518585205, accuracy: 0.666015625
Epoch: 93, step_count: 100, loss: 1.37959623336792, accuracy: 0.578125
Epoch: 93, step_count: 110, loss: 1.2882795333862305, accuracy: 0.6171875
Epoch: 93, step_count: 120, loss: 1.4557383060455322, accuracy: 0.560546875
Epoch: 94, step_count: 0, loss: 1.3646867275238037, accuracy: 0.58984375
Epoch: 94, step_count: 10, loss: 0.9932352304458618, accuracy: 0.716796875
Epoch: 94, step_count: 20, loss: 1.1868088245391846, accuracy: 0.630859375
Epoch: 94, step_count: 30, loss: 1.4077891111373901, accuracy: 0.5703125
Epoch: 94, step_count: 40, loss: 1.2785648107528687, accuracy: 0.607421875
Epoch: 94, step_count: 50, loss: 

Epoch: 5, step_count: 20, loss: 2.808887243270874, accuracy: 0.21484375
Epoch: 5, step_count: 30, loss: 2.7198448181152344, accuracy: 0.23974609375
Epoch: 6, step_count: 0, loss: 2.6701979637145996, accuracy: 0.26416015625
Epoch: 6, step_count: 10, loss: 2.741507053375244, accuracy: 0.248046875
Epoch: 6, step_count: 20, loss: 2.6880321502685547, accuracy: 0.2490234375
Epoch: 6, step_count: 30, loss: 2.5896923542022705, accuracy: 0.2900390625
Epoch: 7, step_count: 0, loss: 2.533294677734375, accuracy: 0.30859375
Epoch: 7, step_count: 10, loss: 2.6030027866363525, accuracy: 0.291015625
Epoch: 7, step_count: 20, loss: 2.5709595680236816, accuracy: 0.275390625
Epoch: 7, step_count: 30, loss: 2.4669926166534424, accuracy: 0.31298828125
Epoch: 8, step_count: 0, loss: 2.401160717010498, accuracy: 0.35009765625
Epoch: 8, step_count: 10, loss: 2.4915897846221924, accuracy: 0.31103515625
Epoch: 8, step_count: 20, loss: 2.473299264907837, accuracy: 0.302734375
Epoch: 8, step_count: 30, loss: 2.38

Epoch: 32, step_count: 30, loss: 1.565030813217163, accuracy: 0.53564453125
Epoch: 33, step_count: 0, loss: 1.3966095447540283, accuracy: 0.57666015625
Epoch: 33, step_count: 10, loss: 1.658334493637085, accuracy: 0.513671875
Epoch: 33, step_count: 20, loss: 1.5667258501052856, accuracy: 0.51611328125
Epoch: 33, step_count: 30, loss: 1.5375500917434692, accuracy: 0.53759765625
Epoch: 34, step_count: 0, loss: 1.3741555213928223, accuracy: 0.5947265625
Epoch: 34, step_count: 10, loss: 1.625962495803833, accuracy: 0.515625
Epoch: 34, step_count: 20, loss: 1.5722439289093018, accuracy: 0.52587890625
Epoch: 34, step_count: 30, loss: 1.5238479375839233, accuracy: 0.54736328125
Epoch: 35, step_count: 0, loss: 1.3390686511993408, accuracy: 0.60546875
Epoch: 35, step_count: 10, loss: 1.6124789714813232, accuracy: 0.52099609375
Epoch: 35, step_count: 20, loss: 1.5333478450775146, accuracy: 0.5302734375
Epoch: 35, step_count: 30, loss: 1.4995471239089966, accuracy: 0.53662109375
Epoch: 36, step_c

Epoch: 60, step_count: 0, loss: 0.9883171916007996, accuracy: 0.7138671875
Epoch: 60, step_count: 10, loss: 1.3183326721191406, accuracy: 0.61767578125
Epoch: 60, step_count: 20, loss: 1.0635133981704712, accuracy: 0.671875
Epoch: 60, step_count: 30, loss: 1.187483310699463, accuracy: 0.64697265625
Epoch: 61, step_count: 0, loss: 0.9491541385650635, accuracy: 0.724609375
Epoch: 61, step_count: 10, loss: 1.2815790176391602, accuracy: 0.62939453125
Epoch: 61, step_count: 20, loss: 1.086298942565918, accuracy: 0.666015625
Epoch: 61, step_count: 30, loss: 1.1599514484405518, accuracy: 0.64404296875
Epoch: 62, step_count: 0, loss: 0.9590537548065186, accuracy: 0.72412109375
Epoch: 62, step_count: 10, loss: 1.2747302055358887, accuracy: 0.62841796875
Epoch: 62, step_count: 20, loss: 1.044909954071045, accuracy: 0.6865234375
Epoch: 62, step_count: 30, loss: 1.125191569328308, accuracy: 0.65478515625
Epoch: 63, step_count: 0, loss: 0.9214721918106079, accuracy: 0.74609375
Epoch: 63, step_count

Epoch: 87, step_count: 10, loss: 0.96671462059021, accuracy: 0.71044921875
Epoch: 87, step_count: 20, loss: 0.7091236710548401, accuracy: 0.79150390625
Epoch: 87, step_count: 30, loss: 0.8560052514076233, accuracy: 0.73291015625
Epoch: 88, step_count: 0, loss: 0.6563557982444763, accuracy: 0.8095703125
Epoch: 88, step_count: 10, loss: 0.9698082208633423, accuracy: 0.7216796875
Epoch: 88, step_count: 20, loss: 0.7224985361099243, accuracy: 0.78662109375
Epoch: 88, step_count: 30, loss: 0.8262861967086792, accuracy: 0.73486328125
Epoch: 89, step_count: 0, loss: 0.6557116508483887, accuracy: 0.82080078125
Epoch: 89, step_count: 10, loss: 0.9260995388031006, accuracy: 0.72119140625
Epoch: 89, step_count: 20, loss: 0.7163698673248291, accuracy: 0.7822265625
Epoch: 89, step_count: 30, loss: 0.8337040543556213, accuracy: 0.74072265625
Epoch: 90, step_count: 0, loss: 0.6516563296318054, accuracy: 0.8232421875
Epoch: 90, step_count: 10, loss: 0.9280698895454407, accuracy: 0.72509765625
Epoch: 9